## **Homework - Session 8**

## **Dataset**

In [1]:
# !wget https://github.com/alexeygrigorev/large-datasets/releases/download/dogs-cats/train.zip

## **Data preparation**

The dataset contains 12,500 images of cats and 12,500 images of dogs. 

Now we need to split this data into train and validation

* Create a `train` and `validation` folders
* In each folder, create `cats` and `dogs` folders
* Move the first 10,000 images to the train folder (from 0 to 9999) for boths cats and dogs - and put them in respective folders
* Move the remaining 2,500 images to the validation folder (from 10000 to 12499)

You can do this manually or with Python (check `os` and `shutil` packages).

In [2]:
# unzip train.zip
!unzip -q '/workspaces/ML_Engineer_ZoomCamp/08_deep-learning/train.zip'

In [6]:
# create folders
# !mkdir val
# !mkdir train/dogs val/dogs train/cats val/cats

In [7]:
# move images
import os

for i in range(0, 10000):
    os.rename(f'train/dog.{i}.jpg', f'train/dogs/dog.{i}.jpg')
    os.rename(f'train/cat.{i}.jpg', f'train/cats/cat.{i}.jpg')

for i in range(10000, 12500):
    os.rename(f'train/dog.{i}.jpg', f'val/dogs/dog.{i}.jpg')
    os.rename(f'train/cat.{i}.jpg', f'val/cats/cat.{i}.jpg')

## **Model**

For this homework we will use Convolutional Neural Network (CNN. Like in the lectures, we'll use Keras.

You need to develop the model with following structure:

* The shape for input should be `(150, 150, 3)`
* Next, create a covolutional layer ([`Conv2D`](https://keras.io/api/layers/convolution_layers/convolution2d/)):
    * Use 32 filters
    * Kernel size should be `(3, 3)` (that's the size of the filter)
    * Use `'relu'` as activation 
* Reduce the size of the feature map with max pooling ([`MaxPooling2D`](https://keras.io/api/layers/pooling_layers/max_pooling2d/))
    * Set the pooling size to `(2, 2)`
* Turn the multi-dimensional result into vectors using a [`Flatten`](https://keras.io/api/layers/reshaping_layers/flatten/) layer
* Next, add a `Dense` layer with 64 neurons and `'relu'` activation
* Finally, create the `Dense` layer with 1 neuron - this will be the output
    * The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use [`SGD`](https://keras.io/api/optimizers/sgd/) with the following parameters:

* `SGD(lr=0.002, momentum=0.8)`


For clarification about kernel size and max pooling, check [Week #11 Office Hours](https://www.youtube.com/watch?v=1WRgdBTUaAc).

### **Libs**

In [ ]:
# !pip3 install numpy matplotlib

In [2]:
# !pip3 install --upgrade tensorflow

In [3]:
# check install
# !python -c "import tensorflow as tf;print(tf.reduce_sum(tf.random.normal([1000, 1000])))"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.optimizers import SGD, Adam
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten